In [ ]:
!pip install chromadb[all]

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.cross_encoder import CrossEncoder
import chromadb
import numpy as np
from typing import List, Dict, Any, Tuple

In [ ]:
class RerankerPipeline:
    def __init__(self,
                 chroma_persist_dir: str = "/content/chroma_db",
                 chroma_collection_name: str = "facom_regulamento",
                 bi_encoder_name: str = "paraphrase-multilingual-MiniLM-L12-v2",
                 cross_encoder_name: str = "cross-encoder/ms-marco-MiniLM-L-6-v2",
                 device: str = "cpu"):
        self.client = chromadb.PersistentClient(path=chroma_persist_dir)
        self.collection = self.client.get_collection(chroma_collection_name)
        self.bi_encoder = SentenceTransformer(bi_encoder_name, device=device)
        self.cross_encoder = CrossEncoder(cross_encoder_name, device=device)

    def retrieve_candidates(self, query: str, top_k: int = 20) -> Dict[str, Any]:
        q_vec = self.bi_encoder.encode([query], convert_to_numpy=True)[0].tolist()
        res = self.collection.query(
            query_embeddings=[q_vec],
            n_results=top_k,
            include=["documents", "distances", "metadatas"]
        )
        ids = res["ids"][0]
        docs = res["documents"][0]
        distances = res["distances"][0]
        metadatas = res.get("metadatas", [None])[0]
        return {"ids": ids, "docs": docs, "distances": distances, "metadatas": metadatas}

    def rerank(self, query: str, docs: List[str]) -> List[Tuple[float, int]]:
        if not docs:
            return []
        pairs = [[query, d] for d in docs]
        scores = self.cross_encoder.predict(pairs)
        indexed = list(enumerate(scores))
        scored = [(float(s), int(i)) for i, s in indexed]
        scored_sorted = sorted(scored, key=lambda x: x[0], reverse=True)
        return scored_sorted

    def retrieve_and_rerank(self, query: str, retrieve_k: int = 20, final_k: int = 3,
                            mix_with_retriever: bool = False, retriever_weight: float = 0.5) -> List[Dict[str, Any]]:
        candidates = self.retrieve_candidates(query, top_k=retrieve_k)
        ids = candidates["ids"]
        docs = candidates["docs"]
        distances = candidates["distances"]
        metadatas = candidates.get("metadatas") or [None] * len(ids)

        if not docs:
            return []

        reranked = self.rerank(query, docs)

        results = []

        if mix_with_retriever and distances:
            d = np.array(distances, dtype=float)
            inv = 1.0 / (1.0 + d)
            if np.ptp(inv) > 0:
                retriever_scores = (inv - inv.min()) / (inv.max() - inv.min())
            else:
                retriever_scores = np.ones_like(inv)
        else:
            retriever_scores = None

        for rerank_score, orig_idx in reranked:
            final_score = rerank_score
            if mix_with_retriever and retriever_scores is not None:
                final_score = (1.0 - retriever_weight) * rerank_score + retriever_weight * float(retriever_scores[orig_idx])
            results.append({
                "id": ids[orig_idx],
                "text": docs[orig_idx],
                "orig_index": orig_idx,
                "retriever_distance": float(distances[orig_idx]) if distances else None,
                "rerank_score": float(rerank_score),
                "final_score": float(final_score),
                "metadata": metadatas[orig_idx] if metadatas else None
            })

        if mix_with_retriever:
            results = sorted(results, key=lambda x: x["final_score"], reverse=True)

        return results[:final_k]


In [ ]:
rp = RerankerPipeline(chroma_persist_dir="/content/chroma_db", device="cpu")
top = rp.retrieve_and_rerank("Entrar no doutorado?", retrieve_k=20, final_k=20)

for i, r in enumerate(top, 1):
    print(f"\n=== Resultado {i} ===")
    print(f"ID: {r['id']}")
    print(f"Final Score: {r['final_score']:.4f}")
    print(f"Text: {r['text'][:2000]}...")


=== Resultado 1 ===
ID: 98e5cba5-12ca-459b-b4ed-f20081814922
Final Score: 0.2599
Text: § 1º Poderão ser aceitas inscrições para o processo seletivo no doutorado sem
a exigência do grau de Mestre, observados os normativos legais da UFMS e da Capes.
 
§ 2º Será permitida a mudança de nível de mestrado para doutorado, de acordo
com as normas estabelecida pela UFMS e pela Capes.
 
10/07/2025, 16:20
SEI/UFMS - 5701360 - Resolução
https://sei.ufms.br/sei/controlador.php?acao=procedimento_trabalhar&acao_origem=procedimento_controlar&acao_retorno=procedimento_cont…
8/21
11/07/2025
N° 8584
Pg. 168...

=== Resultado 2 ===
ID: 41d5d341-e9ac-4c68-a790-7f2d8be995d2
Final Score: -1.3631
Text: atuar como pesquisadores e como professores; e
IV - Doutorado Profissional:  capacitar profissionais qualificados para o exercício
da prática profissional avançada para atender arranjos produtivos em empresas, organizações
públicas e privadas.
 
Parágrafo único. Os Cursos de Mestrado e de Doutorado conduzem ao